In [23]:
#!wget http://files.ipd.uw.edu/pub/RFdiffusion/6f5902ac237024bdd0c176cb93063dc4/Base_ckpt.pt

In [3]:
"""
config_map = {
    "model": config.ModelConfig(),
    "diffuser": config.DiffusionConfig(),
    "preprocess": config.PreprocessConfig(),
    "seq_diffuser": config.SeqDiffusionConfig(),
    
}
IGNORE_KEYS = ["d_time_emb", "d_time_emb_proj", "use_motif_timestep", "prob_self_cond", "str_self_cond"]
def compare_recurse(dict1, dict2, prefix=""):
    for k, v in dict1.items():
        if k in IGNORE_KEYS:
            continue
        if k not in dict2:
            print(f"Missing {prefix} {k}={v} in dict2")
        elif isinstance(v, dict):
            compare_recurse(v, dict2[k], prefix=f"{prefix}_{k}")
        elif v != dict2[k]:
            print(f"Mismatch {prefix} {k}={v} to {dict2[k]} (default)")
    return      

conf_dict = ckpt["config_dict"]
key_diff = set(conf_dict.keys()).difference(set(config_map.keys()))
if any(key_diff):
    print("Missing def for ", key_diff)
    
for k, dataclass_conf in config_map.items():
    if k in conf_dict:
        compare_recurse(conf_dict[k], asdict(dataclass_conf), prefix=k)
"""
print("ignore")

ignore


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from proteome.models.design.rfdiffusion.inference_utils import parse_pdb
from proteome import protein

In [3]:
from proteome.models.folding.openfold.np.relax import cleanup
import io
import numpy as np

In [25]:
from proteome.constants import residue_constants as rc

In [16]:
from proteome.models.design.rfdiffusion.chemical import aa2long, aa2num
from dataclasses import asdict

In [7]:
aa2long_dict = {}
for k,v in aa2num.items():
    aa2long_dict[k] = [atom.strip() if isinstance(atom, str) else "" for atom in list(aa2long[v])[:14]]

In [9]:
for k,v in rc.restype_name_to_atom14_names.items():
    if v == aa2long_dict[k]:
        print("Matched", k)
    else:
        print("Mismatched", k)

Matched ALA
Matched ARG
Matched ASN
Matched ASP
Matched CYS
Matched GLN
Matched GLU
Matched GLY
Matched HIS
Matched ILE
Matched LEU
Matched LYS
Matched MET
Matched PHE
Matched PRO
Matched SER
Matched THR
Matched TRP
Matched TYR
Matched VAL
Mismatched UNK


In [85]:
pdb = "5an7.pdb"

In [105]:
parsed_rfd = parse_pdb(pdb, parse_hetatom=True)

In [187]:
with open(pdb, mode="r") as f:
    pdb_str = f.read()
    
parsed_prtm = protein.from_pdb_string(pdb_str, atom14_format=True, parse_hetatom=True)

In [122]:
from Bio.PDB import PDBParser

In [181]:
for atom in residues[-1]:
    print(atom.coord)

[-20.709  16.209  -6.955]
[-20.563  21.071 -10.339]
[-21.254  15.72   -5.731]
[-22.082  16.548  -4.964]
[-22.654  16.175  -3.779]
[-22.389  17.86   -5.379]
[-21.816  18.272  -6.595]
[-21.009  17.501  -7.371]
[-20.439  17.934  -8.593]
[-20.771  19.238  -8.968]
[-21.605  20.059  -8.174]
[-22.151  19.577  -6.958]
[-20.319  19.857 -10.156]
[-19.633  19.1   -11.205]
[-19.558  19.192 -12.682]
[-18.289  18.518 -13.206]
[-23.01   14.812  -3.702]
[-20.157  15.668  -7.472]
[-21.061  14.857  -5.445]
[-22.938  18.42   -4.879]
[-19.886  17.391  -9.107]
[-21.796  20.926  -8.453]
[-22.706  20.104  -6.429]
[-19.922  18.187 -11.05 ]
[-17.851  18.059 -12.485]
[-23.643  14.607  -4.393]
[-22.226  14.27   -3.815]
[-23.402  14.632  -2.844]
[-18.703  19.124 -10.93 ]
[-18.522  17.888 -13.892]
[-17.699  19.183 -13.568]


In [184]:
residues = []
for chain in model:
    for res in chain:
        residues.append(res)

In [186]:
residues[-1].child_dict

{'C1': <Atom C1>,
 'O1': <Atom O1>,
 'C2': <Atom C2>,
 'C3': <Atom C3>,
 'O3': <Atom O3>,
 'C4': <Atom C4>,
 'C5': <Atom C5>,
 'C6': <Atom C6>,
 'C7': <Atom C7>,
 'C8': <Atom C8>,
 'C9': <Atom C9>,
 'C10': <Atom C10>,
 'C11': <Atom C11>,
 'C12': <Atom C12>,
 'C13': <Atom C13>,
 'C14': <Atom C14>,
 'C31': <Atom C31>,
 'H1': <Atom H1>,
 'H2': <Atom H2>,
 'H4': <Atom H4>,
 'H7': <Atom H7>,
 'H9': <Atom H9>,
 'H10': <Atom H10>,
 'H12': <Atom H12>,
 'H14': <Atom H14>,
 'H1B': <Atom H1B>,
 'H1C': <Atom H1C>,
 'H31': <Atom H31>,
 'H112': <Atom H112>,
 'H14A': <Atom H14A>,
 'H14B': <Atom H14B>}

In [176]:
if residues[-1].id[0].strip():
    print("True")

True


In [161]:
parsed_rfd["info_het"]

[{'idx': 4456, 'atom_id': ' C1 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4457, 'atom_id': ' O1 ', 'atom_type': 'O', 'name': ' LLK'},
 {'idx': 4458, 'atom_id': ' C2 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4459, 'atom_id': ' C3 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4460, 'atom_id': ' O3 ', 'atom_type': 'O', 'name': ' LLK'},
 {'idx': 4461, 'atom_id': ' C4 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4462, 'atom_id': ' C5 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4463, 'atom_id': ' C6 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4464, 'atom_id': ' C7 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4465, 'atom_id': ' C8 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4466, 'atom_id': ' C9 ', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4467, 'atom_id': ' C10', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4468, 'atom_id': ' C11', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4469, 'atom_id': ' C12', 'atom_type': 'C', 'name': ' LLK'},
 {'idx': 4470, 'atom_id': ' C13', 

In [109]:
parsed_prtm.atom_positions.shape

(242, 14, 3)

In [118]:
parsed_prtm.atom_positions[0]

array([[-14.5539999 ,   5.53700018, -21.42200089],
       [-14.38300037,   6.00899982, -20.01499939],
       [-13.97000027,   4.82999992, -19.13199997],
       [-13.85099983,   3.73099995, -19.61800003],
       [-15.7510004 ,   6.56899977, -19.65200043],
       [-16.68899918,   5.64400005, -20.36300087],
       [-16.00399971,   5.34000015, -21.67099953],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ]])

In [119]:
parsed_rfd["xyz"][0]

array([[-14.554,   5.537, -21.422],
       [-14.383,   6.009, -20.015],
       [-13.97 ,   4.83 , -19.132],
       [-13.851,   3.731, -19.618],
       [-15.751,   6.569, -19.652],
       [-16.689,   5.644, -20.363],
       [-16.004,   5.34 , -21.671],
       [  0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   ],
       [  0.   ,   0.   ,   0.   ]], dtype=float32)

In [112]:
np.unique(np.where(parsed_prtm.atom_positions != parsed_rfd["xyz"])[0])

array([  3,  10,  19,  20,  28,  49,  51,  61,  65,  77,  86,  89,  96,
       101, 103, 111, 138, 140, 141, 144, 148, 152, 162, 175, 182, 193,
       202])

In [113]:
np.allclose(parsed_prtm.atom_positions, parsed_rfd["xyz"])

False

In [114]:
np.allclose(parsed_prtm.atom_mask, parsed_rfd["mask"])

True

In [115]:
np.allclose(parsed_prtm.residue_index, parsed_rfd["idx"])

True

In [116]:
np.allclose(parsed_prtm.aatype, parsed_rfd["seq"])

True

In [117]:
list(zip([protein.PDB_CHAIN_IDS[i] for i in parsed_prtm.chain_index], parsed_prtm.residue_index)) == parsed_rfd["pdb_idx"]

True

In [2]:
import re
import os, time, pickle
import torch
from omegaconf import OmegaConf
from omegaconf import DictConfig
import hydra
import logging
import numpy as np
import random
import glob

from proteome.models.design.rfdiffusion.util import writepdb_multi, writepdb
from proteome.models.design.rfdiffusion import inference_utils as iu

In [27]:
from proteome.models.design.rfdiffusion import config
from dataclasses import asdict

In [4]:
def make_deterministic(seed=0):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [5]:
device = torch.cuda.current_device()

In [6]:
make_deterministic()

In [18]:
cfg = config.RFDiffusionConfig(
    inference=config.InferenceConfig(input_pdb="1qys.pdb", model_directory_path=".", num_designs=1, model_runner="default"), 
    contigmap=config.ContigMap(contigs=["5-15/A38-60/30-40"]),
    scaffoldguided=config.ScaffoldGuidedConfig(target_pdb="1qys.pdb")
)
conf = DictConfig(asdict(cfg))

In [19]:
sampler = iu.sampler_selector(conf)

Reading models from .
This is inf_conf.ckpt_path
./Base_ckpt.pt
Assembling -model, -diffuser and -preprocess configs from checkpoint
USING MODEL CONFIG: self._conf[model][n_extra_block] = 4
USING MODEL CONFIG: self._conf[model][n_main_block] = 32
USING MODEL CONFIG: self._conf[model][n_ref_block] = 4
USING MODEL CONFIG: self._conf[model][d_msa] = 256
USING MODEL CONFIG: self._conf[model][d_msa_full] = 64
USING MODEL CONFIG: self._conf[model][d_pair] = 128
USING MODEL CONFIG: self._conf[model][d_templ] = 64
USING MODEL CONFIG: self._conf[model][n_head_msa] = 8
USING MODEL CONFIG: self._conf[model][n_head_pair] = 4
USING MODEL CONFIG: self._conf[model][n_head_templ] = 4
USING MODEL CONFIG: self._conf[model][d_hidden] = 32
USING MODEL CONFIG: self._conf[model][d_hidden_templ] = 32
USING MODEL CONFIG: self._conf[model][p_drop] = 0.15
USING MODEL CONFIG: self._conf[model][SE3_param_full] = {'num_layers': 1, 'num_channels': 32, 'num_degrees': 2, 'n_heads': 4, 'div': 4, 'l0_in_features': 8, '

In [20]:
design_startnum = sampler.inf_conf.design_startnum
if sampler.inf_conf.design_startnum == -1:
    existing = glob.glob(sampler.inf_conf.output_prefix + "*.pdb")
    indices = [-1]
    for e in existing:
        print(e)
        m = re.match(".*_(\d+)\.pdb$", e)
        print(m)
        if not m:
            continue
        m = m.groups()[0]
        indices.append(int(m))
    design_startnum = max(indices) + 1

In [21]:
sampler.inf_conf.num_designs

1

In [22]:
for i_des in range(design_startnum, design_startnum + sampler.inf_conf.num_designs):
    if conf.inference.deterministic:
        make_deterministic(i_des)

    start_time = time.time()
    out_prefix = f"{sampler.inf_conf.output_prefix}_{i_des}"

    x_init, seq_init = sampler.sample_init()
    denoised_xyz_stack = []
    px0_xyz_stack = []
    seq_stack = []
    plddt_stack = []

    x_t = torch.clone(x_init)
    seq_t = torch.clone(seq_init)
    # Loop over number of reverse diffusion time steps.
    for t in range(int(sampler.t_step_input), sampler.inf_conf.final_step - 1, -1):
        px0, x_t, seq_t, plddt = sampler.sample_step(
            t=t, x_t=x_t, seq_init=seq_t, final_step=sampler.inf_conf.final_step
        )
        px0_xyz_stack.append(px0)
        denoised_xyz_stack.append(x_t)
        seq_stack.append(seq_t)
        plddt_stack.append(plddt[0])  # remove singleton leading dimension

    # Flip order for better visualization in pymol
    denoised_xyz_stack = torch.stack(denoised_xyz_stack)
    denoised_xyz_stack = torch.flip(
        denoised_xyz_stack,
        [
            0,
        ],
    )
    px0_xyz_stack = torch.stack(px0_xyz_stack)
    px0_xyz_stack = torch.flip(
        px0_xyz_stack,
        [
            0,
        ],
    )

    # For logging -- don't flip
    plddt_stack = torch.stack(plddt_stack)

    # Save outputs
    os.makedirs(os.path.dirname(out_prefix), exist_ok=True)
    final_seq = seq_stack[-1]

    # Output glycines, except for motif region
    final_seq = torch.where(
        torch.argmax(seq_init, dim=-1) == 21, 7, torch.argmax(seq_init, dim=-1)
    )  # 7 is glycine

    bfacts = torch.ones_like(final_seq.squeeze())
    # make bfact=0 for diffused coordinates
    bfacts[torch.where(torch.argmax(seq_init, dim=-1) == 21, True, False)] = 0
    # pX0 last step
    out = f"{out_prefix}.pdb"

    # Now don't output sidechains
    writepdb(
        out,
        denoised_xyz_stack[0, :, :4],
        final_seq,
        sampler.binderlen,
        chain_idx=sampler.chain_idx,
        bfacts=bfacts,
    )

    # run metadata
    trb = dict(
        config=OmegaConf.to_container(sampler._conf, resolve=True),
        plddt=plddt_stack.cpu().numpy(),
        device=torch.cuda.get_device_name(torch.cuda.current_device())
        if torch.cuda.is_available()
        else "CPU",
        time=time.time() - start_time,
    )
    if hasattr(sampler, "contig_map"):
        for key, value in sampler.contig_map.get_mappings().items():
            trb[key] = value
    with open(f"{out_prefix}.trb", "wb") as f_out:
        pickle.dump(trb, f_out)

    if sampler.inf_conf.write_trajectory:
        # trajectory pdbs
        traj_prefix = (
            os.path.dirname(out_prefix) + "/traj/" + os.path.basename(out_prefix)
        )
        os.makedirs(os.path.dirname(traj_prefix), exist_ok=True)

        out = f"{traj_prefix}_Xt-1_traj.pdb"
        writepdb_multi(
            out,
            denoised_xyz_stack,
            bfacts,
            final_seq.squeeze(),
            use_hydrogens=False,
            backbone_only=False,
            chain_ids=sampler.chain_idx,
        )

        out = f"{traj_prefix}_pX0_traj.pdb"
        writepdb_multi(
            out,
            px0_xyz_stack,
            bfacts,
            final_seq.squeeze(),
            use_hydrogens=False,
            backbone_only=False,
            chain_ids=sampler.chain_idx,
        )

With this beta schedule (linear schedule, beta_0 = 0.04, beta_T = 0.28), alpha_bar_T = 0.00013696050154976547


RuntimeError: one_hot is only applicable to index tensor.

In [11]:
1 + 1

2

In [12]:
import py3Dmol

In [13]:
with open("1qys.pdb", mode="r") as f:
    init_pdb = f.read()

In [14]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(init_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [15]:
with open("./samples/design_0.pdb", mode="r") as f:
    designed_pdb = f.read()

In [16]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(designed_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## Folding

In [17]:
from proteome import protein
from proteome.models.design.proteinmpnn.modeling import ProteinMPNNForSequenceDesign
from proteome.models.folding.omegafold.modeling import OmegaFoldForFolding

In [18]:
sequence_designer = ProteinMPNNForSequenceDesign("vanilla_model-20", random_seed=37)
folder = OmegaFoldForFolding()

In [19]:
target_structure = protein.from_pdb_string(designed_pdb, chain_id="A")
target_structure = protein.to_backbone_only_protein(target_structure)

In [20]:
sequence, score = sequence_designer.design_sequence(target_structure)
print(f"Sequence: {sequence} with global_score {score}")

Sequence: KARKKAIKALKKKIKKANPKKYKISFTCSSKKACKKRAKKYKKKFKGWKKKKKFSKKNGKWVVKLTLKKK with global_score 1.2167978286743164


In [21]:
predicted_protein, confidence = folder.fold(sequence)
folded_pdb = protein.to_pdb(predicted_protein)

In [22]:
PLDDT_BANDS = [
  (0, 50, '#FF7D45'),
  (50, 70, '#FFDB13'),
  (70, 90, '#65CBF3'),
  (90, 100, '#0053D6')
]
view = py3Dmol.view(width=800, height=600)
view.addModelsAsFrames(folded_pdb)

color_map = {i: bands[2] for i, bands in enumerate(PLDDT_BANDS)}
style = {'cartoon': {'colorscheme': {'prop': 'b', 'map': color_map}}}

style['stick'] = {}

view.setStyle({'model': -1}, style)
view.zoomTo()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol